In [8]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import pandas as pd
import plotly.graph_objs as go

In [2]:
df = pd.read_csv('merged_data.csv')

In [3]:
pd.set_option('display.max_columns', None)

In [4]:
df.head()

,Month,DayOfWeek,Carrier,DestAirportID,CRSDepTime,DepDel15,CRSArrTime,ArrDel15,Cancelled,AirportID,Year_y,Day,TimeZone,SkyCondition,Visibility,WeatherType,DryBulbCelsius,WetBulbCelsius,DewPointCelsius,RelativeHumidity,WindSpeed,WindDirection,StationPressure,RecordType,HourlyPrecip,Altimeter
0,4,5,DL,13303,8.0,False,1138,False,False,11433,2013,19,-5,BKN026 OVC047,10.0,-RA,7.2,6.9,6.7,96.0,16.0,220.0,28.75,AA,T,29.45
1,4,5,DL,13303,8.0,False,1138,False,False,11433,2013,19,-5,OVC013,10.0,-RA,7.0,6.6,6.0,93.0,21.0,220.0,28.77,SP,,29.47
2,4,5,DL,11298,8.0,True,1035,True,False,11433,2013,19,-5,BKN026 OVC047,10.0,-RA,7.2,6.9,6.7,96.0,16.0,220.0,28.75,AA,T,29.45
3,4,5,DL,11298,8.0,True,1035,True,False,11433,2013,19,-5,OVC013,10.0,-RA,7.0,6.6,6.0,93.0,21.0,220.0,28.77,SP,,29.47
4,4,5,DL,11292,8.0,False,1008,False,False,11433,2013,19,-5,BKN026 OVC047,10.0,-RA,7.2,6.9,6.7,96.0,16.0,220.0,28.75,AA,T,29.45


In [33]:
df['CRSDepTime'].unique()

array([ 8., 17.,  6., 16., 19., 21., 22., 15., 18., 10.,  7., 20.,  9.,
       12., 14., 13., 11., 24., 23.,  1.,  5.,  3.,  2.,  0.,  4.])

In [39]:
app = dash.Dash(__name__)

# Define la interfaz de usuario con el menú desplegable y el gráfico de barras
app.layout = html.Div([
    html.H1('Gráfico de Barras Dash'),
    dcc.Dropdown(
        id='airport-dropdown',
        options=[
            {'label': 'Todos', 'value': 'all'},  # Opción para seleccionar todos
        ] + [
            {'label': airport_id, 'value': airport_id}
            for airport_id in df['AirportID'].unique()
        ],
        value='all'  # Valor predeterminado: "todos"
    ),
    dcc.Graph(id='bar-chart')
])

# Crea una función de callback para actualizar el gráfico en función de la selección en el menú desplegable
@app.callback(
    Output('bar-chart', 'figure'),
    Input('airport-dropdown', 'value')
)
def actualizar_grafico(airport_id):
    if airport_id == 'all':
        # Si se selecciona "todos", se muestra un gráfico con todos los datos
        df_filtrado = df
        title = 'Cantidad de DepDel15 True en Todos los Aeropuertos'
    else:
        # Filtra el DataFrame en función del valor seleccionado en el menú desplegable
        df_filtrado = df[df['AirportID'] == airport_id]
        title = f'Cantidad de DepDel15 True en el Aeropuerto {airport_id}'

    # Combina Month y Day en una nueva columna para el eje x
    df_filtrado['Month_Day'] = df_filtrado['Month'].astype(str) + '-' + df_filtrado['Day'].astype(str)

    # Agrupa los datos por Month_Day y cuenta la cantidad de DepDel15 True
    data = df_filtrado.groupby('Month_Day')['DepDel15'].sum().reset_index()

    # Crea el gráfico de barras
    fig = {
        'data': [
            {'x': data['Month_Day'], 'y': data['DepDel15'], 'type': 'bar', 'name': 'DepDel15 True'},
        ],
        'layout': {
            'title': title,
            'xaxis': {'title': 'Mes-Día'},
            'yaxis': {'title': 'Cantidad DepDel15 True'},
        }
    }

    return fig

# Ejecuta la aplicación
if __name__ == '__main__':
    app.run_server(debug=True)


In [40]:

app = dash.Dash(__name__)

# Agrega una columna 'Cancelled' en formato numérico (0 o 1) para contar las cancelaciones
df['Cancelled'] = df['Cancelled'].astype(int)

# Define la interfaz de usuario con el menú desplegable, el gráfico de barras y el gráfico de pastel
app.layout = html.Div([
    html.H1('Gráfico de Torta Dash'),
    dcc.Dropdown(
        id='carrier-dropdown',
        options=[
            {'label': carrier, 'value': carrier}
            for carrier in df['Carrier'].unique()
        ],
        value=df['Carrier'].unique()[0]  # Valor predeterminado: el primer valor único de Carrier
    ),
    dcc.Graph(id='pie-chart')
])

# Crea una función de callback para el gráfico de pastel
@app.callback(
    Output('pie-chart', 'figure'),
    Input('carrier-dropdown', 'value')
)
def actualizar_grafico_pastel(carrier):
    # Filtra el DataFrame en función del valor seleccionado en el menú desplegable
    df_filtrado = df[df['Carrier'] == carrier]

    # Calcula la cantidad de vuelos retrasados (DepDel15=True) y no retrasados (DepDel15=False)
    vuelos_retrasados = df_filtrado[df_filtrado['DepDel15'] == True].shape[0]
    vuelos_no_retrasados = df_filtrado[df_filtrado['DepDel15'] == False].shape[0]

    # Crea el gráfico de pastel
    labels = ['Retrasados', 'No Retrasados']
    values = [vuelos_retrasados, vuelos_no_retrasados]

    fig = {
        'data': [
            go.Pie(
                labels=labels,
                values=values,
                textinfo='percent',
            ),
        ],
        'layout': {
            'title': f'Proporción de Vuelos Retrasados y No Retrasados para la Aerolínea {carrier}',
        }
    }

    return fig

# Ejecuta la aplicación
if __name__ == '__main__':
    app.run_server(debug=True)


In [17]:
app = dash.Dash(__name__)

# Agrega una columna 'Cancelled' en formato numérico (0 o 1) para contar las cancelaciones
df['Cancelled'] = df['Cancelled'].astype(int)

# Define la interfaz de usuario con los menús desplegables, el gráfico de barras y el gráfico de pastel
app.layout = html.Div([
    html.H1('Gráfico de Barras y Gráfico de Pastel Dash'),
    html.Label('Selecciona un aeropuerto:'),
    dcc.Dropdown(
        id='airport-dropdown',
        options=[
            {'label': 'Todos', 'value': 'all'},  # Opción para seleccionar todos los aeropuertos
        ] + [
            {'label': airport_id, 'value': airport_id}
            for airport_id in df['AirportID'].unique()
        ],
        value='all'  # Valor predeterminado: "todos"
    ),
    html.Label('Selecciona una aerolínea:'),
    dcc.Dropdown(
        id='carrier-dropdown',
        options=[
            {'label': 'Todos', 'value': 'all'},  # Opción para seleccionar todas las aerolíneas
        ] + [
            {'label': carrier, 'value': carrier}
            for carrier in df['Carrier'].unique()
        ],
        value='all'  # Valor predeterminado: "todos"
    ),
    dcc.Graph(id='bar-chart'),
    dcc.Graph(id='pie-chart')
])

# Crea una función de callback para actualizar el gráfico de barras en función de la selección en el menú desplegable de aeropuertos
@app.callback(
    Output('bar-chart', 'figure'),
    Input('airport-dropdown', 'value')
)
def actualizar_grafico_aeropuertos(airport_id):
    if airport_id == 'all':
        # Si se selecciona "todos", se muestra un gráfico con todos los datos
        df_filtrado = df
        title = 'Cantidad de DepDel15 True en Todos los Aeropuertos'
    else:
        # Filtra el DataFrame en función del valor seleccionado en el menú desplegable de aeropuertos
        df_filtrado = df[df['AirportID'] == airport_id]
        title = f'Cantidad de DepDel15 True en el Aeropuerto {airport_id}'

    # Combina Month y Day en una nueva columna para el eje x
    df_filtrado['Month_Day'] = df_filtrado['Month'].astype(str) + '-' + df_filtrado['Day'].astype(str)

    # Agrupa los datos por Month_Day y cuenta la cantidad de DepDel15 True
    data = df_filtrado.groupby('Month_Day')['DepDel15'].sum().reset_index()

    # Crea el gráfico de barras
    fig = {
        'data': [
            {'x': data['Month_Day'], 'y': data['DepDel15'], 'type': 'bar', 'name': 'DepDel15 True'},
        ],
        'layout': {
            'title': title,
            'xaxis': {'title': 'Mes-Día'},
            'yaxis': {'title': 'Cantidad DepDel15 True'},
        }
    }

    return fig

# Crea una función de callback para el gráfico de pastel en función de la selección en el menú desplegable de aerolíneas
@app.callback(
    Output('pie-chart', 'figure'),
    Input('carrier-dropdown', 'value')
)
def actualizar_grafico_aerolineas(carrier):
    if carrier == 'all':
        # Si se selecciona "todos", se muestra un gráfico con todos los datos
        df_filtrado = df
        title = 'Proporción de Vuelos Retrasados y No Retrasados en Todas las Aerolíneas'
    else:
        # Filtra el DataFrame en función del valor seleccionado en el menú desplegable de aerolíneas
        df_filtrado = df[df['Carrier'] == carrier]
        title = f'Proporción de Vuelos Retrasados y No Retrasados para la Aerolínea {carrier}'

    # Calcula la cantidad de vuelos retrasados (DepDel15=True) y no retrasados (DepDel15=False)
    vuelos_retrasados = df_filtrado[df_filtrado['DepDel15'] == True].shape[0]
    vuelos_no_retrasados = df_filtrado[df_filtrado['DepDel15'] == False].shape[0]

    # Crea el gráfico de pastel
    labels = ['Retrasados', 'No Retrasados']
    values = [vuelos_retrasados, vuelos_no_retrasados]

    fig = {
        'data': [
            go.Pie(
                labels=labels,
                values=values,
                textinfo='percent',
            ),
        ],
        'layout': {
            'title': title,
        }
    }

    return fig

# Ejecuta la aplicación
if __name__ == '__main__':
    app.run_server(debug=True)



In [41]:
app = dash.Dash(__name__)

# Agrega una columna 'Cancelled' en formato numérico (0 o 1) para contar las cancelaciones
df['Cancelled'] = df['Cancelled'].astype(int)

# Define la interfaz de usuario con los menús desplegables, los gráficos de barras y el gráfico de pastel
app.layout = html.Div([
    html.H1('Gráfico de Barras y Gráfico de Pastel Dash'),
    html.Div([
        html.Div([
            html.Label('Selecciona un aeropuerto:'),
            dcc.Dropdown(
                id='airport-dropdown',
                options=[
                    {'label': 'Todos', 'value': 'all'},  # Opción para seleccionar todos los aeropuertos
                ] + [
                    {'label': airport_id, 'value': airport_id}
                    for airport_id in df['AirportID'].unique()
                ],
                value='all'  # Valor predeterminado: "todos"
            ),
        ], className="four columns"),
        html.Div([
            html.Label('Selecciona una aerolínea:'),
            dcc.Dropdown(
                id='carrier-dropdown',
                options=[
                    {'label': 'Todos', 'value': 'all'},  # Opción para seleccionar todas las aerolíneas
                ] + [
                    {'label': carrier, 'value': carrier}
                    for carrier in df['Carrier'].unique()
                ],
                value='all'  # Valor predeterminado: "todos"
            ),
        ], className="four columns"),
    ], className="row"),
    html.Div([
        html.Div([
            dcc.Graph(id='bar-chart')
        ], className="six columns"),
        html.Div([
            dcc.Graph(id='pie-chart')
        ], className="six columns"),
    ], className="row"),
])

# Crea una función de callback para actualizar el gráfico de barras en función de la selección en el menú desplegable de aeropuertos
@app.callback(
    Output('bar-chart', 'figure'),
    Input('airport-dropdown', 'value')
)
def actualizar_grafico_aeropuertos(airport_id):
    if airport_id == 'all':
        # Si se selecciona "todos", se muestra un gráfico con todos los datos
        df_filtrado = df
        title = 'Cantidad de DepDel15 True en Todos los Aeropuertos'
    else:
        # Filtra el DataFrame en función del valor seleccionado en el menú desplegable de aeropuertos
        df_filtrado = df[df['AirportID'] == airport_id]
        title = f'Cantidad de DepDel15 True en el Aeropuerto {airport_id}'

    # Combina Month y Day en una nueva columna para el eje x
    df_filtrado['Month_Day'] = df_filtrado['Month'].astype(str) + '-' + df_filtrado['Day'].astype(str)

    # Agrupa los datos por Month_Day y cuenta la cantidad de DepDel15 True
    data = df_filtrado.groupby('Month_Day')['DepDel15'].sum().reset_index()

    # Crea el gráfico de barras
    fig = {
        'data': [
            {'x': data['Month_Day'], 'y': data['DepDel15'], 'type': 'bar', 'name': 'DepDel15 True'},
        ],
        'layout': {
            'title': title,
            'xaxis': {'title': 'Mes-Día'},
            'yaxis': {'title': 'Cantidad DepDel15 True'},
        }
    }

    return fig

# Crea una función de callback para el gráfico de pastel en función de la selección en el menú desplegable de aerolíneas
@app.callback(
    Output('pie-chart', 'figure'),
    Input('carrier-dropdown', 'value')
)
def actualizar_grafico_aerolineas(carrier):
    if carrier == 'all':
        # Si se selecciona "todos", se muestra un gráfico con todos los datos
        df_filtrado = df
        title = 'Proporción de Vuelos Retrasados y No Retrasados en Todas las Aerolíneas'
    else:
        # Filtra el DataFrame en función del valor seleccionado en el menú desplegable de aerolíneas
        df_filtrado = df[df['Carrier'] == carrier]
        title = f'Proporción de Vuelos Retrasados y No Retrasados para la Aerolínea {carrier}'

    # Calcula la cantidad de vuelos retrasados (DepDel15=True) y no retrasados (DepDel15=False)
    vuelos_retrasados = df_filtrado[df_filtrado['DepDel15'] == True].shape[0]
    vuelos_no_retrasados = df_filtrado[df_filtrado['DepDel15'] == False].shape[0]

    # Crea el gráfico de pastel
    labels = ['Retrasados', 'No Retrasados']
    values = [vuelos_retrasados, vuelos_no_retrasados]

    fig = {
        'data': [
            go.Pie(
                labels=labels,
                values=values,
                textinfo='percent',
            ),
        ],
        'layout': {
            'title': title,
        }
    }

    return fig

# Ejecuta la aplicación
if __name__ == '__main__':
    app.run_server(debug=True)


In [22]:
app = dash.Dash(__name__)

# Agrega una columna 'Cancelled' en formato numérico (0 o 1) para contar las cancelaciones
df['Cancelled'] = df['Cancelled'].astype(int)

# Define la interfaz de usuario con los menús desplegables, los gráficos de barras y el gráfico de pastel
app.layout = html.Div([
    html.H1('Gráfico de Barras y Gráfico de Pastel Dash'),
    html.Div([
        html.Div([
            html.Label('Selecciona un aeropuerto:'),
            dcc.Dropdown(
                id='airport-dropdown',
                options=[
                    {'label': 'Todos', 'value': 'all'},  # Opción para seleccionar todos los aeropuertos
                ] + [
                    {'label': airport_id, 'value': airport_id}
                    for airport_id in df['AirportID'].unique()
                ],
                value='all'  # Valor predeterminado: "todos"
            ),
        ], className="four columns"),
        html.Div([
            html.Label('Selecciona una aerolínea:'),
            dcc.Dropdown(
                id='carrier-dropdown',
                options=[
                    {'label': 'Todos', 'value': 'all'},  # Opción para seleccionar todas las aerolíneas
                ] + [
                    {'label': carrier, 'value': carrier}
                    for carrier in df['Carrier'].unique()
                ],
                value='all'  # Valor predeterminado: "todos"
            ),
        ], className="four columns"),
    ], className="row"),
    html.Div([
        html.Div([
            dcc.Graph(id='bar-chart')
        ], className="six columns"),
        html.Div([
            dcc.Graph(id='pie-chart')
        ], className="six columns"),
    ], className="row"),
])

# Crea una función de callback para actualizar los gráficos en función de las selecciones en los menús desplegables
@app.callback(
    [Output('bar-chart', 'figure'), Output('pie-chart', 'figure')],
    [Input('airport-dropdown', 'value'), Input('carrier-dropdown', 'value')]
)
def actualizar_graficos(airport_id, carrier):
    if airport_id == 'all':
        # Si se selecciona "todos" en el menú de aeropuertos, se muestra un gráfico con todos los datos
        df_filtrado = df
        title_bar = 'Cantidad de DepDel15 True en Todos los Aeropuertos'
    else:
        # Filtra el DataFrame en función del valor seleccionado en el menú de aeropuertos
        df_filtrado = df[df['AirportID'] == airport_id]
        title_bar = f'Cantidad de DepDel15 True en el Aeropuerto {airport_id}'

    if carrier == 'all':
        # Si se selecciona "todos" en el menú de aerolíneas, se muestra un gráfico con todos los datos
        title_pie = 'Proporción de Vuelos Retrasados y No Retrasados en Todas las Aerolíneas'
    else:
        # Filtra el DataFrame en función del valor seleccionado en el menú de aerolíneas
        df_filtrado = df_filtrado[df_filtrado['Carrier'] == carrier]
        title_pie = f'Proporción de Vuelos Retrasados y No Retrasados para la Aerolínea {carrier}'

    # Combina Month y Day en una nueva columna para el eje x
    df_filtrado['Month_Day'] = df_filtrado['Month'].astype(str) + '-' + df_filtrado['Day'].astype(str)

    # Agrupa los datos por Month_Day y cuenta la cantidad de DepDel15 True
    data_bar = df_filtrado.groupby('Month_Day')['DepDel15'].sum().reset_index()

    # Crea el gráfico de barras
    fig_bar = {
        'data': [
            {'x': data_bar['Month_Day'], 'y': data_bar['DepDel15'], 'type': 'bar', 'name': 'DepDel15 True'},
        ],
        'layout': {
            'title': title_bar,
            'xaxis': {'title': 'Mes-Día'},
            'yaxis': {'title': 'Cantidad DepDel15 True'},
        }
    }

    # Calcula la cantidad de vuelos retrasados (DepDel15=True) y no retrasados (DepDel15=False)
    vuelos_retrasados = df_filtrado[df_filtrado['DepDel15'] == True].shape[0]
    vuelos_no_retrasados = df_filtrado[df_filtrado['DepDel15'] == False].shape[0]

    # Crea el gráfico de pastel
    labels_pie = ['Retrasados', 'No Retrasados']
    values_pie = [vuelos_retrasados, vuelos_no_retrasados]

    fig_pie = {
        'data': [
            go.Pie(
                labels=labels_pie,
                values=values_pie,
                textinfo='percent',
            ),
        ],
        'layout': {
            'title': title_pie,
        }
    }

    return fig_bar, fig_pie

# Ejecuta la aplicación
if __name__ == '__main__':
    app.run_server(debug=True)


In [32]:
# Agrega una columna 'Cancelled' en formato numérico (0 o 1) para contar las cancelaciones
df['Cancelled'] = df['Cancelled'].astype(int)

# Calcula el promedio de los valores para crear una tabla de calor
heat_map_data = df.groupby(['DayOfWeek', 'CRSDepTime'])['WindSpeed'].mean().reset_index()

# Crea la figura de la tabla de calor
heatmap_fig = go.Figure(data=go.Heatmap(
    z=heat_map_data['WindSpeed'],
    x=heat_map_data['CRSDepTime'],
    y=heat_map_data['DayOfWeek'],
    colorscale='YlGnBu',  # Escoge la paleta de colores que prefieras
))

app = dash.Dash(__name__)

# Define la interfaz de usuario con los gráficos y la tabla de calor
app.layout = html.Div([
    html.H1('Gráfico de Barras, Gráfico de Pastel y Tabla de Calor Dash'),
    html.Div([
        html.Div([
            html.Label('Selecciona un aeropuerto:'),
            dcc.Dropdown(
                id='airport-dropdown',
                options=[
                    {'label': 'Todos', 'value': 'all'},  # Opción para seleccionar todos los aeropuertos
                ] + [
                    {'label': airport_id, 'value': airport_id}
                    for airport_id in df['AirportID'].unique()
                ],
                value='all'  # Valor predeterminado: "todos"
            ),
        ], className="four columns"),
        html.Div([
            html.Label('Selecciona una aerolínea:'),
            dcc.Dropdown(
                id='carrier-dropdown',
                options=[
                    {'label': 'Todos', 'value': 'all'},  # Opción para seleccionar todas las aerolíneas
                ] + [
                    {'label': carrier, 'value': carrier}
                    for carrier in df['Carrier'].unique()
                ],
                value='all'  # Valor predeterminado: "todos"
            ),
        ], className="four columns"),
    ], className="row"),
    html.Div([
        html.Div([
            dcc.Graph(id='bar-chart')
        ], className="six columns"),
        html.Div([
            dcc.Graph(id='pie-chart')
        ], className="six columns"),
    ], className="row"),
    dcc.Graph(id='heat-map', figure=heatmap_fig)  # Agrega la tabla de calor
])

# Crea una función de callback para actualizar los gráficos en función de las selecciones en los menús desplegables
@app.callback(
    [Output('bar-chart', 'figure'), Output('pie-chart', 'figure')],
    [Input('airport-dropdown', 'value'), Input('carrier-dropdown', 'value')]
)
def actualizar_graficos(airport_id, carrier):
    if airport_id == 'all':
        # Si se selecciona "todos" en el menú de aeropuertos, se muestra un gráfico con todos los datos
        df_filtrado = df
        title_bar = 'Cantidad de DepDel15 True en Todos los Aeropuertos'
    else:
        # Filtra el DataFrame en función del valor seleccionado en el menú de aeropuertos
        df_filtrado = df[df['AirportID'] == airport_id]
        title_bar = f'Cantidad de DepDel15 True en el Aeropuerto {airport_id}'

    if carrier == 'all':
        # Si se selecciona "todos" en el menú de aerolíneas, se muestra un gráfico con todos los datos
        title_pie = 'Proporción de Vuelos Retrasados y No Retrasados en Todas las Aerolíneas'
    else:
        # Filtra el DataFrame en función del valor seleccionado en el menú de aerolíneas
        df_filtrado = df_filtrado[df_filtrado['Carrier'] == carrier]
        title_pie = f'Proporción de Vuelos Retrasados y No Retrasados para la Aerolínea {carrier}'

    # Combina Month y Day en una nueva columna para el eje x
    df_filtrado['Month_Day'] = df_filtrado['Month'].astype(str) + '-' + df_filtrado['Day'].astype(str)

    # Agrupa los datos por Month_Day y cuenta la cantidad de DepDel15 True
    data_bar = df_filtrado.groupby('Month_Day')['DepDel15'].sum().reset_index()

    # Crea el gráfico de barras
    fig_bar = {
        'data': [
            {'x': data_bar['Month_Day'], 'y': data_bar['DepDel15'], 'type': 'bar', 'name': 'DepDel15 True'},
        ],
        'layout': {
            'title': title_bar,
            'xaxis': {'title': 'Mes-Día'},
            'yaxis': {'title': 'Cantidad DepDel15 True'},
        }
    }

    # Calcula la cantidad de vuelos retrasados (DepDel15=True) y no retrasados (DepDel15=False)
    vuelos_retrasados = df_filtrado[df_filtrado['DepDel15'] == True].shape[0]
    vuelos_no_retrasados = df_filtrado[df_filtrado['DepDel15'] == False].shape[0]

    # Crea el gráfico de pastel
    labels_pie = ['Retrasados', 'No Retrasados']
    values_pie = [vuelos_retrasados, vuelos_no_retrasados]

    fig_pie = {
        'data': [
            go.Pie(
                labels=labels_pie,
                values=values_pie,
                textinfo='percent',
            ),
        ],
        'layout': {
            'title': title_pie,
        }
    }

    return fig_bar, fig_pie

# Ejecuta la aplicación
if __name__ == '__main__':
    app.run_server(debug=True)

In [38]:
# Agrega una columna 'Cancelled' en formato numérico (0 o 1) para contar las cancelaciones
df['Cancelled'] = df['Cancelled'].astype(int)

# Calcula el promedio de los valores para crear una tabla de calor
heat_map_data = df.groupby(['DayOfWeek', 'CRSDepTime'])['WindSpeed'].mean().reset_index()

# Crea la figura de la tabla de calor
heatmap_fig = go.Figure(data=go.Heatmap(
    z=heat_map_data['WindSpeed'],
    x=heat_map_data['CRSDepTime'],
    y=heat_map_data['DayOfWeek'],
    colorscale='YlGnBu',  # Escoge la paleta de colores que prefieras
))

app = dash.Dash(__name__)

# Define la interfaz de usuario con los gráficos, la tabla de calor y el menú desplegable para seleccionar el valor a promediar
app.layout = html.Div([
    html.H1('Gráfico de Barras, Gráfico de Pastel y Tabla de Calor Dash'),
    html.Div([
        html.Div([
            html.Label('Selecciona un aeropuerto:'),
            dcc.Dropdown(
                id='airport-dropdown',
                options=[
                    {'label': 'Todos', 'value': 'all'},  # Opción para seleccionar todos los aeropuertos
                ] + [
                    {'label': airport_id, 'value': airport_id}
                    for airport_id in df['AirportID'].unique()
                ],
                value='all'  # Valor predeterminado: "todos"
            ),
        ], className="four columns"),
        html.Div([
            html.Label('Selecciona una aerolínea:'),
            dcc.Dropdown(
                id='carrier-dropdown',
                options=[
                    {'label': 'Todos', 'value': 'all'},  # Opción para seleccionar todas las aerolíneas
                ] + [
                    {'label': carrier, 'value': carrier}
                    for carrier in df['Carrier'].unique()
                ],
                value='all'  # Valor predeterminado: "todos"
            ),
        ], className="four columns"),
    ], className="row"),
    html.Div([
        html.Div([
            dcc.Graph(id='bar-chart')
        ], className="six columns"),
        html.Div([
            dcc.Graph(id='pie-chart')
        ], className="six columns"),
    ], className="row"),
    dcc.Graph(id='heat-map', figure=heatmap_fig),  # Agrega la tabla de calor
    html.Div([
        html.Label('Selecciona un valor para promediar:'),
        dcc.Dropdown(
            id='valor-a-promediar-dropdown',
            options=[
                {'label': 'WindSpeed', 'value': 'WindSpeed'},
                {'label': 'RelativeHumidity', 'value': 'RelativeHumidity'},
                {'label': 'StationPressure', 'value': 'StationPressure'},
            ],
            value='WindSpeed'  # Valor predeterminado: WindSpeed
        ),
    ], style={'width': '30%', 'margin': 'auto'}),  # Centrar el menú desplegable
])

# Crea una función de callback para actualizar la tabla de calor en función de la selección en el menú desplegable del valor a promediar
@app.callback(
    Output('heat-map', 'figure'),
    Input('valor-a-promediar-dropdown', 'value')
)
def actualizar_tabla_de_calor(valor_a_promediar):
    # Calcula el promedio del valor seleccionado
    heat_map_data = df.groupby(['DayOfWeek', 'CRSDepTime'])[valor_a_promediar].mean().reset_index()

    # Crea la figura de la tabla de calor actualizada
    heatmap_fig = go.Figure(data=go.Heatmap(
        z=heat_map_data[valor_a_promediar],
        x=heat_map_data['CRSDepTime'],
        y=heat_map_data['DayOfWeek'],
        colorscale='YlGnBu',  # Escoge la paleta de colores que prefieras
    ))

    return heatmap_fig

if __name__ == '__main__':
    app.run_server(debug=True)


In [37]:
# Agrega una columna 'Cancelled' en formato numérico (0 o 1) para contar las cancelaciones
df['Cancelled'] = df['Cancelled'].astype(int)

# Calcula el promedio de los valores para crear una tabla de calor
heat_map_data = df.groupby(['DayOfWeek', 'CRSDepTime'])['WindSpeed'].mean().reset_index()

# Crea la figura de la tabla de calor
heatmap_fig = go.Figure(data=go.Heatmap(
    z=heat_map_data['WindSpeed'],
    x=heat_map_data['CRSDepTime'],
    y=heat_map_data['DayOfWeek'],
    colorscale='YlGnBu',  # Escoge la paleta de colores que prefieras
))

# Inicializa la aplicación de Dash
app = dash.Dash(__name__)

# Define la interfaz de usuario con los gráficos, la tabla de calor y el menú desplegable para seleccionar el valor a promediar
app.layout = html.Div([
    html.H1('Gráfico de Barras, Gráfico de Pastel y Tabla de Calor Dash'),
    html.Div([
        html.Div([
            html.Label('Selecciona un valor para promediar:'),
            dcc.Dropdown(
                id='valor-a-promediar-dropdown',
                options=[
                    {'label': 'WindSpeed', 'value': 'WindSpeed'},
                    {'label': 'RelativeHumidity', 'value': 'RelativeHumidity'},
                    {'label': 'StationPressure', 'value': 'StationPressure'},
                ],
                value='WindSpeed'  # Valor predeterminado: WindSpeed
            ),
        ], style={'width': '30%', 'margin': 'auto'}),  # Centrar el menú desplegable
    ], className="row"),
    html.Div([
        dcc.Graph(id='heat-map', figure=heatmap_fig),  # Agrega la tabla de calor
    ], className="row"),
    html.Div([
        html.Label(id='heatmap-title')
    ]),
    html.Div([
        html.Div([
            html.Label('Selecciona un aeropuerto:'),
            dcc.Dropdown(
                id='airport-dropdown',
                options=[
                    {'label': 'Todos', 'value': 'all'},  # Opción para seleccionar todos los aeropuertos
                ] + [
                    {'label': airport_id, 'value': airport_id}
                    for airport_id in df['AirportID'].unique()
                ],
                value='all'  # Valor predeterminado: "todos"
            ),
        ], className="four columns"),
        html.Div([
            html.Label('Selecciona una aerolínea:'),
            dcc.Dropdown(
                id='carrier-dropdown',
                options=[
                    {'label': 'Todos', 'value': 'all'},  # Opción para seleccionar todas las aerolíneas
                ] + [
                    {'label': carrier, 'value': carrier}
                    for carrier in df['Carrier'].unique()
                ],
                value='all'  # Valor predeterminado: "todos"
            ),
        ], className="four columns"),
    ], className="row"),
    html.Div([
        html.Div([
            dcc.Graph(id='bar-chart')
        ], className="six columns"),
        html.Div([
            dcc.Graph(id='pie-chart')
        ], className="six columns"),
    ], className="row"),
])

# Crea una función de callback para actualizar el título de la tabla de calor y la tabla de calor en función de la selección en el menú desplegable del valor a promediar
@app.callback(
    [Output('heat-map', 'figure'), Output('heatmap-title', 'children')],
    Input('valor-a-promediar-dropdown', 'value')
)
def actualizar_tabla_de_calor(valor_a_promediar):
    # Calcula el promedio del valor seleccionado
    heat_map_data = df.groupby(['DayOfWeek', 'CRSDepTime'])[valor_a_promediar].mean().reset_index()

    # Crea la figura de la tabla de calor actualizada
    heatmap_fig = go.Figure(data=go.Heatmap(
        z=heat_map_data[valor_a_promediar],
        x=heat_map_data['CRSDepTime'],
        y=heat_map_data['DayOfWeek'],
        colorscale='YlGnBu',  # Escoge la paleta de colores que prefieras
    ))

    title = f'Mapa de calor por Hora y Día del campo {valor_a_promediar}'

    return heatmap_fig, title

if __name__ == '__main__':
    app.run_server(debug=True)
